In [ ]:
!pip install -U gensim

     |████████████████████████████████| 24.1 MB 1.4 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
import pandas as pd
import numpy as np
import gensim
from sklearn.linear_model import LogisticRegression
from gensim.models.doc2vec import TaggedDocument
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.svm import SVC
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
import multiprocessing
from sklearn import utils
from sklearn.model_selection import train_test_split
from gensim.parsing.preprocessing import remove_stopwords
from gensim.utils import simple_preprocess
from gensim.parsing.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import re
import nltk

import itertools
from gensim.similarities import MatrixSimilarity
from gensim.parsing.preprocessing import preprocess_documents
from gensim.parsing.preprocessing import preprocess_string

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
resumeDf = pd.read_csv('resume_with_skills.csv')
resumeDf.drop(resumeDf.columns[[0]], axis = 1, inplace = True)
resumeDf.tail(5)

,id,category,resume,cleaned_resume,skills
436,437,java developer,GRADUATE RESEARCH ASSISTANT Pro...,graduate research assistant professional profi...,"['remote control', 'analytics', 'java', 'frame..."
437,438,java developer,GRADUATE RESEARCH ASSISTANT ...,graduate research assistant summary obtain int...,"['cryptographic protocol', 'linux', 'authentic..."
438,439,java developer,STUDENT ASSISTANT Qualificati...,student assistant qualification algorithm java...,"['linux', 'authentication', 'android studio', ..."
439,440,java developer,YW TEAM LEAD Summary ...,yw team lead summary motivated team lead posse...,"['linux', 'code review', 'java', 'react', 'sup..."
440,441,java developer,SENIOR APPLICATION ADMINISTRATOR & SO...,senior application administrator solution arch...,"['linux', 'java', 'data file', 'shell', 'asp n..."


In [ ]:
jdDf = pd.read_csv('jd_with_skills.csv')
jdDf.drop(jdDf.columns[[0]], axis = 1, inplace = True)
jdDf.tail(5)

,id,category,jd,cleaned_jd,skills
501,502,java developer,Synechron is a recognized leader and expert in...,synechron recognized leader expert building bu...,"['design', 'information management', 'web serv..."
502,503,java developer,Note: For further information please contact S...,note information please contact sonal hatila s...,"['angular', 'oracle', 'framework', 'java', 'sp..."
503,504,java developer,"Synechron on behalf of our client, a global le...",synechron behalf client global leader distribu...,"['security', 'design', 'message queue', 'infor..."
504,505,java developer,Please contactPatrick TrainorCanyon Associates...,please contactpatrick trainorcanyon associates...,"['design', 'database', 'testing', 'java', 'sup..."
505,506,java developer,Java Developer – Algorithmic Trading - NYCDura...,java developer algorithmic trading nycduration...,"['shell', 'java']"


In [ ]:
from scipy import spatial
import ast
import math 

def get_cosine(a, b):
  result = 1 - spatial.distance.cosine(a, b)
  return result

def get_sim(given, required):
  score = 0
  given = ast.literal_eval(given)
  required = ast.literal_eval(required)
  given = set(given)
  for x in required:
    if x in given:
      score = score + 1
  result = 0
  if len(required) != 0:
    result = ((score*1.0)/len(required))
  return result

def merge_skills(df, ids):

  x = []

  for i in ids:
    for y in ast.literal_eval(df['skills'][i-1]):
      x.append(y)
  
  return set(x)

In [ ]:
tagged_corpus = []
for i in range(len(jdDf)):
  tagged_corpus.append(TaggedDocument(jdDf['cleaned_jd'][i].split(' '), [jdDf['id'][i]]))

model = Doc2Vec(tagged_corpus,
                dm=0, 
                vector_size=500, 
                window=5, 
                min_count=1, 
                epochs=10, 
                hs=1)

In [ ]:
model.save('./d2vJD.model')

In [ ]:
tempDf = pd.DataFrame()
tempDf['id'] = ""
tempDf['vectors'] = ""
tempDf

,id,vectors


In [ ]:
for i in range(len(jdDf)):
  id = jdDf['id'][i]
  doc = jdDf['cleaned_jd'][i].split(' ')
  jd_vector = model.infer_vector(doc)

  tempDf.loc[len(tempDf.index)] = [id, jd_vector]

tempDf

,id,vectors
0,1,"[0.075227976, -0.4130248, -0.2778718, -0.26840..."
1,2,"[-0.040785845, -0.08826695, 0.29435351, -0.494..."
2,3,"[0.18924971, -0.23298241, -0.01197488, -0.3338..."
3,4,"[0.19030201, -0.4971362, -0.2807913, -0.240542..."
4,5,"[0.15016913, -0.3379973, 0.061276678, -0.09453..."
...,...,...
501,502,"[0.01012734, -0.23584718, -0.11456097, -0.3569..."
502,503,"[0.1731841, -0.33140472, 0.02143297, -0.014503..."
503,504,"[-0.013596261, -0.297683, 0.06740245, -0.29811..."
504,505,"[-0.10225573, -0.25365448, -0.073201805, -0.12..."


In [ ]:
matchDf = pd.DataFrame()
matchDf['id'] = ""
matchDf['matched_ids'] = ""
matchDf['merged_skills'] = ""

for i in range(len(jdDf)):
  doc = jdDf['cleaned_jd'][i].split(' ')
  jd_vector = model.infer_vector(doc)

  temp2Df = pd.DataFrame();
  temp2Df['id'] = ""
  temp2Df['cosine'] = ""
  for j in range(len(tempDf)):

    if i == j:
      continue

    second_vector = tempDf['vectors'][j]
    cosine = get_cosine(jd_vector, second_vector)
    
    temp2Df.loc[len(temp2Df.index)] = [tempDf['id'][j], cosine]

  
  temp2Df.sort_values(by=['cosine'], ascending = False, inplace = True)

  listt = []
  for inedx, row in temp2Df.head(10).iterrows():
    listt.append(row['id'])

  print(i)
  # print(temp2Df.head(10))
  # print(listt)
  # print(merge_skills(jdDf, listt))
  matchDf.loc[len(matchDf.index)] = [jdDf['id'][i], listt, merge_skills(jdDf, listt)]

matchDf  

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
matchDf

,id,matched_ids,merged_skills
0,1,"[427.0, 407.0, 147.0, 143.0, 128.0, 158.0, 402...","{component, monitoring, domain knowledge, data..."
1,2,"[6.0, 488.0, 483.0, 41.0, 463.0, 475.0, 16.0, ...","{component, business, ajax, apache tomcat, jav..."
2,3,"[36.0, 20.0, 46.0, 43.0, 31.0, 50.0, 5.0, 33.0...","{openshift, monitoring, specification, busines..."
3,4,"[98.0, 41.0, 51.0, 101.0, 250.0, 478.0, 88.0, ...","{monitoring, data analysis, specification, bus..."
4,5,"[31.0, 3.0, 50.0, 33.0, 46.0, 23.0, 40.0, 30.0...","{component, monitoring, specification, busines..."
...,...,...,...
501,502,"[504.0, 487.0, 230.0, 498.0, 3.0, 32.0, 477.0,...","{component, business, machine learning, design..."
502,503,"[455.0, 463.0, 497.0, 487.0, 496.0, 459.0, 467...","{business, junit, ajax, medium, mode, design, ..."
503,504,"[502.0, 487.0, 93.0, 461.0, 396.0, 453.0, 442....","{metadata, business, mode, hbase, machine lear..."
504,505,"[372.0, 488.0, 299.0, 463.0, 25.0, 483.0, 340....","{component, monitoring, specification, busines..."


In [ ]:
matchDf.to_csv('jd_similar.csv', index=False)